In [2]:
import matplotlib
matplotlib.use('agg')

import os
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
from keras import layers

import keras
from pylab import plt
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, figure
from netCDF4 import Dataset
import h5py

from dask.diagnostics import ProgressBar
ProgressBar().register()
import netCDF4

In [3]:
#Saved path
outdir='C:/Users/user/Research/Research Code/CNN/trainning/world predicted/'
ifile_data ='C:/Users/user/Research/Research Code/CNN/Data/'
testsizepath='C:/Users/user/Research/Research Code/CNN/y_test/y_testsize_1428_89_180_1.nc'
y_test_size= xr.open_dataarray(testsizepath)

In [4]:
os.system('mkdir -p '+outdir)
N_gpu = 0
print('open inputdata')

open inputdata


In [9]:
#Imported data
ds = xr.open_dataset(ifile_data+'nonnormalized_ersst_1854_2022.nc')
ds

<xarray.Dataset>
Dimensions:  (lev: 1, lat: 89, lon: 180, time: 2028)
Coordinates:
  * lev      (lev) float64 0.0
  * lat      (lat) float32 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-12-01
Data variables:
    sst      (time, lat, lon, lev) float32 ...
Attributes: (12/37)
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    description:               In situ data: ICOADS2.5 before 2007 and NCEP i...
    keywords_vocabulary:       NASA Global Change Master Directory (GCMD) Sci...
    keywords:                  Earth Science > Oceans > Ocean Temperature > S...
    instrument:                Conventional thermometers
    source_comment:            SSTs were observed by conventional thermometer...
    ...                        ...
    creator_url_original:      https://www.ncei.noaa.gov
    license:                   No constraints on data access or use
    comment:                   SSTs were observed by conventional thermometer...
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    dataset_title:             NOAA Extended Reconstructed SST V5
    data_modified:             2023-01-04

In [10]:
##Prepared data
ds=ds['sst'].data 
ds[ds<0]=0 #treat nan values as 0
np.nan_to_num(ds,copy=False)
ds=ds/(np.nanmax(ds)) 
dates=pd.date_range(start='1854-01-01',periods=len(ds))
label=np.array(dates.month)
label=label-1

In [11]:
##Data separation

N_skip = 0
N_train = 288
N_gab_1 = 0
N_test = 1116

x = ds

In [12]:
##Verify that we have sufficient information for the requirements.
if N_skip+N_train+N_gab_1+N_test > x.shape[0]:
    raise Exception('not enough timesteps in input file!')

x = x.astype('float32')
x = x[:N_skip+N_train+N_gab_1+N_test]

lat,lon,lev=x.shape[1:4]

In [13]:
def prepare_data(lead_time):
    ''' split up data in predictor and predictant set by shifting
     it according to the given lead time, and then split up
     into train, developement and test set'''
    if lead_time == 0:
        X = x
        y = X[:]

    else:

        X = x[:]
        y = x[:]
    

    X_train = X[N_skip-lead_time:N_skip+N_train-lead_time]
    y_train = y[N_skip-lead_time:N_skip+N_train-lead_time]
    
    X_test = X[N_skip+N_train+N_gab_1:N_skip+N_train+N_gab_1+N_test]
    y_test = y[N_skip+N_train+N_gab_1:N_skip+N_train+N_gab_1+N_test]
    
       
    return X_train,y_train, X_test, y_test

In [14]:
# fixed (not-tuned params)
batch_size = 32
num_epochs = 10
pool_size = 1
drop_prob=0
conv_activation='relu'

In [15]:
def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])

In [16]:
def build_model(conv_depth, kernel_size, hidden_size, n_hidden_layers, lr):

    model = keras.Sequential([
            
            ## Convolution which involves dimensionality reduction (similar to Encoder in an autoencoder)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation, input_shape=(lat,lon,lev)),
            layers.MaxPooling2D(pool_size=pool_size),
            Dropout(drop_prob),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size=pool_size),
            
            # end "encoder"
            
            
            # dense layers (Automatic flattening and reshaping occurs.)
            ] + [layers.Dense(hidden_size, activation='sigmoid') for i in range(n_hidden_layers)] +
             
            [
            
            
            # start "Decoder" (upsampling of the encoder above)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            layers.Convolution2D(lev, kernel_size, padding='same', activation=None)
            ]
            )
    
    
    optimizer= keras.optimizers.Adam(lr=lr)

    if N_gpu > 1:
        with tf.device("/cpu:0"):
            # convert the model to a model that can be trained with N_GPU GPUs
             model = keras.utils.multi_gpu_model(model, gpus=N_gpu)
             
    model.compile(loss='mean_squared_error', optimizer = optimizer)
    
    return model

In [18]:
##Make prediction
for lead_time in range(0,1):
    X_train,y_train, X_test, y_test = prepare_data(lead_time)
    
    ## The tuning process produced the hyperparameters below.
    params = {'conv_depth': 32, 'hidden_size': 500,
              'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}

    
    print(params)
    param_string = '_'.join([str(e) for e in (N_train,num_epochs,lead_time)])
    
    
    #run the model
    model = build_model(**params)
    
    print(model.summary())
    
    
    print('start training')
    hist = model.fit(X_train, y_train,
                       batch_size = batch_size,
             verbose=1, 
             epochs = num_epochs,
             validation_data=(X_test,y_test),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights.h5', monitor='val_loss', 
                                                    verbose=1, save_best_only=True, 
                                                    save_weights_only=True, mode='auto', period=1)]
             )
    
    print('finished training')
    
    # To ensure the best model performance from the training based on learning curve,
    # Because the early stopping callback saves the model's "patience" epochs after the best one, this is required.
    model.load_weights('best_weights.h5')
    
    # delete the file generated by Model Checkppoint
    os.system('rm best_weights.h5')
    
    
    model.save_weights(outdir+'weights_tunedparams_leadtime_sst_1878_2022_'+str(lead_time)+'params_'+param_string+'.h5')
        
        
    # reformat history
        
    hist =  hist.history

    
    
    y_test_predicted = model.predict(X_test)

    # compute accuracy
    res = []
    rmse = np.sqrt(np.mean((y_test_predicted - y_test)**2))
    acc = acc_score(y_test_predicted, y_test)

    res.append(dict(hist=hist,params=params, scores=[rmse,acc]))

    pickle.dump(res,open(outdir+'training_result_sequential_sst_1878_2022_'+param_string+'.pkl','wb'))
    
    # plot loss value
    plt.figure()
    plt.plot(hist['val_loss'], label='val_loss')
    plt.plot(hist['loss'], label='train loss')
    
    plt.legend()
    
    #save loss to .png
    plt.savefig(outdir+'cwcnn_history_tunedparams_leadtime_sst_1878_2022_'+str(lead_time)+'.png')

    pd.DataFrame(hist).to_csv(outdir+'history_tunedparams_leadtime_sst_1878_2022_'+str(lead_time)+'.csv')


    '''y_test is a xarray dataarray, but y_test_predicted is now a numpy array.
    Therefore, we convert it to an xarray with exactly the same coordinates and dims by using dim from y_test.'''
    # save the validation
    y_test_predicted_new = xr.DataArray(data=y_test_predicted, coords=y_test_size.coords, dims=y_test_size.dims)
    y_test_new = xr.DataArray(data=y_test, coords=y_test_size.coords, dims=y_test_size.dims)
    
    # save the predictions
    y_test_predicted_new.to_netcdf(outdir+'/predictions_tuned_leadtime_worldmap_sst_1878_2022_'+str(lead_time)+'params_'+param_string+'.nc')
    y_test_new.to_netcdf(outdir+'/truevalues_tuned_leadtime_worldmap_sst_1878_2022_'+str(lead_time)+'params_'+param_string+'.nc')

{'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 89, 180, 32)       544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 180, 32)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 89, 180, 32)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 89, 180, 32)       16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 89, 180, 32)      0         
 2D)                                                             
                              

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - ETA: 0s - loss: 0.0777
Epoch 1: val_loss improved from inf to 0.06251, saving model to best_weights.h5
9/9 [==============================] - 38s 4s/step - loss: 0.0777 - val_loss: 0.0625
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 0.0517
Epoch 2: val_loss improved from 0.06251 to 0.04032, saving model to best_weights.h5
9/9 [==============================] - 44s 5s/step - loss: 0.0517 - val_loss: 0.0403
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.0327
Epoch 3: val_loss improved from 0.04032 to 0.02482, saving model to best_weights.h5
9/9 [==============================] - 43s 5s/step - loss: 0.0327 - val_loss: 0.0248
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.0198
Epoch 4: val_loss improved from 0.02482 to 0.01489, saving model to best_weights.h5
9/9 [==============================] - 42s 5s/step - loss: 0.0198 - val_loss: 0.0149
Epoch 5/10
9/9 [==============================]

In [19]:
#Result Analysis
import pickle
import json

import numpy as np

res = pickle.load(open(outdir+'training_result_sequential_sst_1878_2022_288_10_0.pkl','rb'))

# get each run's final validating loss

final_val_losses = [e['hist']['val_loss'][-1] for e in res]

# convert to array
final_val_losses = np.array(final_val_losses)

# find the smallest index.

idx_smallest = np.argmin(final_val_losses)


res_best = res[idx_smallest]

print(res_best)

with open('training_result.txt','w') as f :
    f.write(str(res_best))

{'hist': {'loss': [0.07769640535116196, 0.05173170194029808, 0.03273507580161095, 0.0198227446526289, 0.012503856793045998, 0.010042145848274231, 0.00964202731847763, 0.00906306691467762, 0.008342709392309189, 0.007703348994255066], 'val_loss': [0.06251122802495956, 0.040316738188266754, 0.024818498641252518, 0.014892845414578915, 0.010559113696217537, 0.009777812287211418, 0.009408846497535706, 0.00865725614130497, 0.00799910444766283, 0.007350979372859001]}, 'params': {'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}, 'scores': [0.08573787, 0.9470993240876397]}


# END